In [1]:
import grid2op
import numpy as np

from superposition_theorem import State

env = grid2op.make("l2rpn_case14_sandbox")
param = env.parameters
param.ENV_DC = True  # force the computation of the powerflow in DC mode
param.MAX_LINE_STATUS_CHANGED = 99999
param.MAX_SUB_CHANGED = 99999
param.NO_OVERFLOW_DISCONNECTION = True
env.change_parameters(param)
env.change_forecast_parameters(param)

# Line disconnection

Here we use the "State" interface to compute the disconnection (at its creation) and then we can see the results of each n-k (which are a lot)

First, let's initialize the "State" with the unary line disconnections we want to combine

In [2]:
obs = env.reset()
state = State.from_grid2op_obs(obs, line_ids_disc_unary=(1, 2, 3, 4, 7))

And now we can compute the flows after the disconnection of 2 powerlines (for example): 

In [3]:
p_or_12 = state.compute_flows_disco_lines((1, 2))

And now we can check the results compared to using a real simulator:

In [4]:
obs_true, *_ = obs.simulate(env.action_space({"set_bus": {"lines_or_id": [(1, -1), (2, -1)]}}), time_step=0)
por_true = obs_true.p_or
print(f"Max difference: {np.abs(p_or_12 - por_true).max():.2e} MW")

Max difference: 4.02e-06 MW


# Line reconnection

In this second part, we show how we could reconnect powerline. For this, we first generate a grid state with some line disconnected (1 and 2 for the example) and then see how we can leverage the superposition theorem to reconnect it.

In [5]:
obs = env.reset()
start_obs, *_  = env.step(env.action_space({"set_bus": {"lines_or_id": [(1, -1), (2, -1)]}}))
state = State.from_grid2op_obs(start_obs, line_ids_reco_unary=(1, 2))

Now we can reconnect these powerlines using the superposition theorem:

In [6]:
p_or_12 = state.compute_flows_reco_lines((1, 2))

And finally, we can check the results

In [7]:
obs_true, *_ = start_obs.simulate(env.action_space({"set_bus": {"lines_or_id": [(1, 1), (2, 1)]}}), time_step=0)
por_true = obs_true.p_or
print(f"Max difference: {np.abs(p_or_12 - por_true).max():.2e} MW")

Max difference: 1.91e-06 MW


# Topology modification (node splitting)

In this section we demonstrate how to use the "State" api to perform topological modifications using the extended superposition theorem

In [8]:
start_obs = env.reset()

state = State.from_grid2op_obs(start_obs)

Then I build the action (which is a more complicated process, this is why we first build a "state" without any single actions, and then rebuild a state afterwards)

The first action is:

In [9]:
# first unary action
id_sub1 = 5
act1 = state.get_emptyact()
act1.set_subid(id_sub1)
# act1.get_elem_sub()  # to know which elements are connected to this substation
act1.set_bus(lines_id=[(7, 1), (8, 1), (9, 2), (17, 2)],
             loads_id=[(4, 2)],
             gens_id=[(2, 1), (3, 2)])

# second unary action
id_sub2 = 4
act2 = state.get_emptyact()
act2.set_subid(id_sub2)
# act2.get_elem_sub()  # to know which elements are connected to this substation
act2.set_bus(lines_id=[(1, 2), (4, 1), (6, 2), (17, 1)],
             loads_id=[(3, 2)])

Then we can compute the state with both unary actions :

In [10]:
state.add_unary_actions_grid2op(start_obs, subs_actions_unary=[act1, act2])

In [11]:
p_or_sub1sub2 = state.compute_flows_node_split((act1, act2))

And finally we can check the consistency of the results:

In [12]:
obs_true, *_ = start_obs.simulate(act1.to_grid2op(env.action_space) + act2.to_grid2op(env.action_space), time_step=0)
por_true = obs_true.p_or
print(f"Max difference: {np.abs(p_or_sub1sub2 - por_true).max():.2e} MW")

Max difference: 1.91e-06 MW


# Combination of different action types (ex. line disconnection and topology)

In this section we show how to use the "state" api to combine unary action of different types (for example line disconenction and topology). We will reuse the same kind of action as the one showed in the above sections.

TODO section in progress !

In [13]:
start_obs = env.reset()

state = State.from_grid2op_obs(start_obs,
                               line_ids_disc_unary=(2, 3),
                               subs_actions_unary=[act1, act2])


In [14]:
p_or_combined = state.compute_flows(line_ids_disc=(2, 3), subs_actions=[act1, act2])

NotImplementedError: TODO

In [ ]:
combined_act = act1.to_grid2op(env.action_space) + act2.to_grid2op(env.action_space)
combined_act += env.action_space({"set_line_status": [(2, -1), (3, -1)]})
obs_true, r, done, info = start_obs.simulate(combined_act, time_step=0)
assert not info["exception"], f'Error while performing the powerflow check: {info["exception"]}'
por_true = obs_true.p_or
print(f"Max difference: {np.abs(p_or_combined - por_true).max():.2e} MW")

In [ ]:
p_or_combined